# Import Packages 

In [23]:
#where environment is located
import sys
print(sys.executable)


/home/denizgokalp.5/code/mitchell5/neo_bank/part_1-data_exploration/.venv/bin/python


In [24]:
import pandas as pd
import great_expectations as gx
import os
import numpy as np

# Importing Data and Cleaning it

## Devices

In [25]:
devices=pd.read_csv("/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/devices.csv")
devices.head()

string_field_0 string_field_1
0          Apple      user_6809
1          Apple      user_3636
2          Apple       user_510
3          Apple      user_3468
4          Apple      user_7036

In [26]:
len(devices)

19431

In [27]:
devices=devices.drop_duplicates()

In [28]:
len(devices)

19431

In [29]:
devices.describe()

string_field_0 string_field_1
count           19431          19431
unique              4          19431
top           Android      user_6809
freq             9714              1

In [30]:
devices.rename(columns={
    "string_field_0": "device_type",
    "string_field_1": "user_id"
},inplace=True)

In [31]:
devices['user_id'] = devices['user_id'].str.lstrip('user_')

In [32]:
devices['device_type'].unique()

array(['Apple', 'brand', 'Android', 'Unknown'], dtype=object)

In [33]:
#since we don't know what "brand" is, let's change it to "unknown" as well
devices["device_type"] = devices["device_type"].replace("brand", "Unknown")


In [34]:
devices['device_type'].unique()

array(['Apple', 'Unknown', 'Android'], dtype=object)

In [35]:
# Check if all user_id s are numeric
mask = devices['user_id'].str.isnumeric()
all_numeric = mask.all()
all_numeric

False

In [36]:
#  Drop rows where user_id is NaN
devices['user_id'] = pd.to_numeric(devices['user_id'], errors='coerce')
devices = devices.dropna(subset=['user_id'])

In [37]:
len(devices)

19430

In [38]:
# Convert to integer if all values are now valid
devices['user_id'] = devices['user_id'].astype('Int64')

In [39]:
#checking that all user_ids are a standard length
lengths = devices["user_id"].astype(str).str.len()

min_len = lengths.min()
max_len = lengths.max()

print(f"Min length: {min_len}")
print(f"Max length: {max_len}")


Min length: 1
Max length: 5


In [40]:
print("User IDs with the shortest length:")
print(devices[devices["user_id"].astype(str).str.len() == min_len]["user_id"].unique())

User IDs with the shortest length:
<IntegerArray>
[1, 6, 4, 0, 5, 3, 7, 9, 8, 2]
Length: 10, dtype: Int64


In [41]:
print("User IDs with the longest length:")
print(devices[devices["user_id"].astype(str).str.len() == max_len]["user_id"].unique())

User IDs with the longest length:
<IntegerArray>
[11885, 15533, 11964, 10670, 19220, 14587, 15593, 12295, 12617, 18270,
 ...
 13121, 15790, 10442, 15105, 10984, 13531, 12568, 12296, 19056, 12199]
Length: 9430, dtype: Int64


In [42]:
devices.head()

device_type  user_id
0       Apple     6809
1       Apple     3636
2       Apple      510
3       Apple     3468
4       Apple     7036

In [43]:
#checks for any empty strings
(devices == '').sum()


device_type    0
user_id        0
dtype: Int64

In [44]:
#checks for any nulls
devices.isnull().sum()

device_type    0
user_id        0
dtype: int64

In [47]:
#creates new cleaned parquet file
devices.to_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/devices.parquet", index=False)

## Transactions

In [49]:
import pandas as pd

# Load CSV
df = pd.read_csv("/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/transactions.csv")

# Save as Parquet
df.to_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/transactions.parquet", engine="pyarrow", index=False)

In [50]:
# very big file, will use parquet
parquet_path = "/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/transactions.parquet"

trans = pd.read_parquet(parquet_path)

In [51]:
trans.head()

transaction_id transactions_type transactions_currency  amount_usd  \
0     transaction_1884            REFUND                   AED        1.76   
1     transaction_1728            REFUND                   AED      639.31   
2  transaction_1755782            REFUND                   AED        3.49   
3     transaction_1973             TOPUP                   AED     6079.39   
4     transaction_1716             TOPUP                   AED      206.65   

  transactions_state ea_cardholderpresence  ea_merchant_mcc ea_merchant_city  \
0          COMPLETED                  None              NaN             None   
1          COMPLETED                  None              NaN             None   
2          COMPLETED                  None              NaN             None   
3          COMPLETED                  None              NaN             None   
4          COMPLETED                  None              NaN             None   

  ea_merchant_country direction     user_id                    created_date  
0                None   INBOUND   user_8098  2018-09-25 16:03:40.978243 UTC  
1                None   INBOUND    user_182  2018-03-31 13:45:25.262231 UTC  
2                None   INBOUND  user_14271  2018-11-17 16:47:25.748975 UTC  
3                None   INBOUND   user_4773  2018-08-13 21:51:28.670363 UTC  
4                None   INBOUND   user_1363  2018-10-30 14:34:57.479895 UTC

In [52]:
len(trans)

2740075

In [53]:
trans=trans.drop_duplicates()

In [54]:
len(trans)

2740075

In [55]:
trans.describe()

amount_usd  ea_merchant_mcc
count  2.740075e+06     1.581417e+06
mean   1.703225e+05     5.699736e+03
std    9.634014e+07     9.493660e+02
min    0.000000e+00     7.420000e+02
25%    2.030000e+00     5.411000e+03
50%    8.510000e+00     5.812000e+03
75%    2.822000e+01     5.921000e+03
max    8.510645e+10     9.406000e+03

In [56]:
trans['transaction_id'] = trans['transaction_id'].str.lstrip('transaction_')

In [57]:
trans['user_id'] = trans['user_id'].str.lstrip('user_')

In [58]:
trans['transactions_type'].unique()

array(['REFUND', 'TOPUP', 'EXCHANGE', 'TRANSFER', 'CARD_PAYMENT',
       'CARD_REFUND', 'ATM', 'FEE', 'CASHBACK', 'TAX'], dtype=object)

In [60]:
trans['transactions_currency'].unique()

array(['AED', 'AUD', 'BCH', 'BGN', 'BTC', 'CAD', 'CHF', 'CZK', 'DKK',
       'ETH', 'EUR', 'GBP', 'HKD', 'HRK', 'HUF', 'ILS', 'INR', 'JPY',
       'LTC', 'MAD', 'MXN', 'NOK', 'NZD', 'PLN', 'QAR', 'RON', 'RUB',
       'SAR', 'SEK', 'SGD', 'THB', 'TRY', 'USD', 'XRP', 'ZAR'],
      dtype=object)

In [61]:
trans['amount_usd'].max()

85106453190.03

In [62]:
trans['amount_usd'].min()

0.0

In [63]:
trans.dtypes

transaction_id            object
transactions_type         object
transactions_currency     object
amount_usd               float64
transactions_state        object
ea_cardholderpresence     object
ea_merchant_mcc          float64
ea_merchant_city          object
ea_merchant_country       object
direction                 object
user_id                   object
created_date              object
dtype: object

In [64]:
trans['created_date'] = pd.to_datetime(trans['created_date'],errors='coerce').dt.tz_localize(None)

In [65]:
len(trans['created_date'])

2740075

In [66]:
#removes rows with null created_date value
trans = trans.dropna(subset=['created_date'])

In [67]:
trans.dtypes

transaction_id                   object
transactions_type                object
transactions_currency            object
amount_usd                      float64
transactions_state               object
ea_cardholderpresence            object
ea_merchant_mcc                 float64
ea_merchant_city                 object
ea_merchant_country              object
direction                        object
user_id                          object
created_date             datetime64[ns]
dtype: object

In [68]:
trans['transactions_state'].unique()

array(['COMPLETED', 'CANCELLED', 'DECLINED', 'PENDING', 'REVERTED',
       'FAILED'], dtype=object)

In [69]:
trans['ea_cardholderpresence'] = trans['ea_cardholderpresence'].astype('bool')

In [70]:
trans['ea_cardholderpresence'].unique()

array([False,  True])

In [71]:
trans['ea_merchant_mcc'].unique()[:5]
#drop

array([  nan, 9222., 4111., 4112., 5651.])

In [72]:
uv = trans['ea_merchant_city'].unique()[:5]
print(list(uv))
#drop

[None, 'Dubai', 'tfl.gov.uk/cp', 'Stockholm', 'M?nchen-flugh']


In [73]:
trans['ea_merchant_country'].unique()


array([None, 'ARE', 'GBR', 'SWE', 'DEU', 'FRA', 'EST', 'NLD', 'POL',
       'ISL', 'ESP', 'USA', 'LUX', 'DNK', 'IRL', 'JOR', 'ROM', 'ITA',
       'LTU', 'AUS', 'HKG', 'SGP', 'NZL', 'FJI', 'IDN', 'CHE', 'MYS',
       'CHN', 'VNM', 'BGR', 'MNE', 'NOR', 'ISR', 'BEL', 'CYP', 'CZE',
       'ROU', 'GIB', 'EGY', 'PRT', 'CAN', 'ZAF', 'MEX', 'COL', 'PER',
       'ARG', 'HUN', 'LKA', 'BOL', 'BRA', 'LVA', 'THA', 'RUS', 'FIN',
       'AUT', 'HRV', 'GRC', 'JPN', 'REU', 'SRB', 'BIH', 'CHL', 'TTO',
       'LBN', 'IND', 'KAZ', 'CRI', 'TWN', 'TUR', 'URY', 'AND', 'QAT',
       'LIE', 'BHS', 'MAR', 'MUS', 'OMN', 'MLT', 'BMU', 'UGA', 'ARM',
       'SYC', 'PHL', 'KEN', 'MCO', 'MDV', 'UZB', 'GEO', 'ETH', 'MOZ',
       'TCA', 'NPL', 'JAM', 'CUB', 'LAO', 'UKR', 'KHM', 'SVN', 'VCT',
       'SVK', 'BLR', 'PAN', 'MMR', 'DOM', 'TZA', 'ECU', 'MKD', 'VAT',
       'HND', 'GTM', 'PYF', 'KOR', 'SXM', 'MTQ', 'PRI', 'GLP', 'MNG',
       'TUN', 'NGA', 'AGO', 'VEN', 'SAU', 'NER', 'ATG', 'MDA', 'MYT',
       'PRY', 'SEN', 

In [74]:
# Replace empty strings with NaN (null)
trans.replace('', np.nan, inplace=True)

In [75]:
trans['ea_merchant_country'] = trans['ea_merchant_country'].fillna("UNKNOWN")

In [76]:
trans['direction'].unique()


array(['INBOUND', 'OUTBOUND'], dtype=object)

In [77]:
#drop some columns we don't need
trans = trans.drop(["ea_merchant_city", "ea_merchant_mcc"], axis=1)


In [78]:
# convert id fields to integer as well
trans['transaction_id'] = trans['transaction_id'].astype('int64')
trans['user_id'] = trans['user_id'].astype('int64')

In [79]:
#convert amount_usd to float
trans['amount_usd'] = trans['amount_usd'].astype('float')

In [80]:
trans.dtypes

transaction_id                    int64
transactions_type                object
transactions_currency            object
amount_usd                      float64
transactions_state               object
ea_cardholderpresence              bool
ea_merchant_country              object
direction                        object
user_id                           int64
created_date             datetime64[ns]
dtype: object

In [81]:
#checks for any nulls
trans.isnull().sum()

transaction_id           0
transactions_type        0
transactions_currency    0
amount_usd               0
transactions_state       0
ea_cardholderpresence    0
ea_merchant_country      0
direction                0
user_id                  0
created_date             0
dtype: int64

In [82]:
#checks for any empty strings
(trans == '').sum()

transaction_id           0
transactions_type        0
transactions_currency    0
amount_usd               0
transactions_state       0
ea_cardholderpresence    0
ea_merchant_country      0
direction                0
user_id                  0
created_date             0
dtype: int64

In [83]:
#creates new clean file
trans.to_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/transactions.parquet", index=False)

## Users

In [84]:
users=pd.read_csv("/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/users.csv")
users.head()

user_id  birth_year country city                    created_date  \
0   user_2811        1988      IE    C  2018-03-04 21:46:39.616834 UTC   
1   user_4750        1987      FR   Ry  2018-04-07 06:49:18.345736 UTC   
2  user_17686        1984      GB  Ayr  2018-12-04 10:28:02.653147 UTC   
3  user_18779        1965      GB  Ayr  2018-12-31 07:42:19.353613 UTC   
4   user_7823        1999      PL  Buk  2018-06-07 03:22:40.590209 UTC   

   user_settings_crypto_unlocked      plan  \
0                              0  STANDARD   
1                              0  STANDARD   
2                              0   PREMIUM   
3                              0  STANDARD   
4                              0  STANDARD   

   attributes_notifications_marketing_push  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   attributes_notifications_marketing_email  num_contacts  num_referrals  \
0                                       NaN             0              0   
1                                       NaN             0              0   
2                                       NaN             6              0   
3                                       NaN             0              0   
4                                       NaN             0              0   

   num_successful_referrals  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0

In [85]:
len(users)

19430

In [86]:
users=users.drop_duplicates()

In [87]:
len(users)

19430

In [88]:
users['user_id'] = users['user_id'].str.lstrip('user_')

In [89]:
# converting date field
users['created_date'] = pd.to_datetime(users['created_date'],errors='coerce').dt.tz_localize(None)

In [90]:
# convert to int64
users['user_id'] = pd.to_numeric(users['user_id'], errors='coerce').astype('Int64')

In [91]:
users.dtypes

user_id                                              Int64
birth_year                                           int64
country                                             object
city                                                object
created_date                                datetime64[ns]
user_settings_crypto_unlocked                        int64
plan                                                object
attributes_notifications_marketing_push            float64
attributes_notifications_marketing_email           float64
num_contacts                                         int64
num_referrals                                        int64
num_successful_referrals                             int64
dtype: object

In [92]:
users['birth_year'].min()

1929

In [93]:
users['birth_year'].max()

2001

In [94]:
users.drop(columns={'city'},inplace=True)

In [95]:
users['plan'].unique()

array(['STANDARD', 'PREMIUM', 'METAL', 'METAL_FREE', 'PREMIUM_OFFER',
       'PREMIUM_FREE'], dtype=object)

In [96]:
users['user_settings_crypto_unlocked'].unique()

array([0, 1])

In [97]:
users['user_settings_crypto_unlocked'] = users['user_settings_crypto_unlocked'].astype('boolean')

In [98]:
users['attributes_notifications_marketing_push'].unique()

array([nan,  1.,  0.])

In [99]:
users['attributes_notifications_marketing_push'] = users['attributes_notifications_marketing_push'].round().astype('boolean')


In [100]:
users['attributes_notifications_marketing_email'].unique()

array([nan,  0.,  1.])

In [101]:
users['attributes_notifications_marketing_email'] = users['attributes_notifications_marketing_email'].astype('boolean')


In [102]:
users['num_contacts'].min()

0

In [103]:
users['num_contacts'].max()

2918

In [104]:
users['num_referrals'].unique()

array([0])

In [105]:
users['num_referrals'].min()

0

In [106]:
users['num_referrals'].max()

0

In [107]:
users.drop(columns={'num_referrals'},inplace=True)

In [108]:
users['num_successful_referrals'].min()

0

In [109]:
users['num_successful_referrals'].max()

0

In [110]:
users.drop(columns={'num_successful_referrals'},inplace=True)

In [111]:
users['country'].unique()

array(['IE', 'FR', 'GB', 'PL', 'ES', 'NO', 'BE', 'SI', 'IT', 'RO', 'CH',
       'DE', 'CZ', 'HU', 'GR', 'NL', 'MT', 'SE', 'PT', 'LT', 'FI', 'LV',
       'EE', 'AT', 'CY', 'HR', 'LU', 'BG', 'DK', 'GG', 'GP', 'JE', 'AU',
       'SK', 'LI', 'IM', 'IS', 'GI', 'MQ', 'GF', 'RE'], dtype=object)

In [112]:
users['created_date'].min()

Timestamp('2018-01-01 08:42:24.799709')

In [113]:
users['created_date'].max()

Timestamp('2019-01-03 07:34:36.638892')

In [114]:
#check for nulls
users.isnull().sum()/len(users)

user_id                                     0.000000
birth_year                                  0.000000
country                                     0.000000
created_date                                0.000000
user_settings_crypto_unlocked               0.000000
plan                                        0.000000
attributes_notifications_marketing_push     0.340196
attributes_notifications_marketing_email    0.340196
num_contacts                                0.000000
dtype: float64

In [115]:
#check for empty strings
(users=='').sum()

user_id                                     0
birth_year                                  0
country                                     0
created_date                                0
user_settings_crypto_unlocked               0
plan                                        0
attributes_notifications_marketing_push     0
attributes_notifications_marketing_email    0
num_contacts                                0
dtype: Int64

In [116]:
#creates new clean file
users.to_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/users.parquet", index=False)

## Notifications

In [117]:
notifs=pd.read_csv("/home/denizgokalp.5/code/mitchell5/neo_bank/data/raw/notifications.csv")
notifs.head()

reason channel status    user_id  \
0  METAL_RESERVE_PLAN     SMS   SENT  user_4703   
1  METAL_RESERVE_PLAN     SMS   SENT  user_2397   
2  METAL_RESERVE_PLAN     SMS   SENT  user_2411   
3  METAL_RESERVE_PLAN     SMS   SENT  user_1119   
4  METAL_RESERVE_PLAN     SMS   SENT  user_3499   

                     created_date  
0  2018-10-14 01:48:13.319987 UTC  
1  2018-10-17 08:07:05.709072 UTC  
2  2018-10-14 03:46:34.244392 UTC  
3  2018-10-14 14:16:35.801185 UTC  
4  2018-10-14 18:41:09.268675 UTC

In [118]:
len(notifs)

121813

In [119]:
notifs=notifs.drop_duplicates()

In [120]:
len(notifs)

121813

In [121]:
# cleaning user_id
notifs['user_id'] = notifs['user_id'].str.lstrip('user_')

In [122]:
# converting date field
notifs['created_date'] = pd.to_datetime(notifs['created_date'],errors='coerce').dt.tz_localize(None)

In [123]:
notifs['reason'].unique()

array(['METAL_RESERVE_PLAN', 'REENGAGEMENT_ACTIVE_FUNDS', 'BLACK_FRIDAY',
       'LOST_CARD_ORDER', 'FIFTH_PAYMENT_PROMO', 'JOINING_ANNIVERSARY',
       'PUMPKIN_PAYMENT_NOTIFICATION', 'PREMIUM_ENGAGEMENT_FEES_SAVED',
       'PREMIUM_ENGAGEMENT_INACTIVE_CARD', 'WELCOME_HOME',
       'METAL_GAME_START', 'NO_INITIAL_CARD_USE', 'NO_INITIAL_CARD_ORDER',
       'ONBOARDING_TIPS_ACTIVATED_USERS',
       'ENGAGEMENT_SPLIT_BILL_RESTAURANT',
       'MADE_MONEY_REQUEST_NOT_SPLIT_BILL',
       'NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER'], dtype=object)

In [124]:
notifs['channel'].unique()

array(['SMS', 'PUSH', 'EMAIL'], dtype=object)

In [125]:
notifs['status'].unique()

array(['SENT', 'FAILED'], dtype=object)

In [126]:
notifs.dtypes

reason                  object
channel                 object
status                  object
user_id                 object
created_date    datetime64[ns]
dtype: object

In [127]:
notifs['created_date'] = pd.to_datetime(notifs['created_date'],errors='coerce').dt.tz_localize(None)

In [128]:
#checking that all user_ids are a standard length
lengths = notifs["user_id"].astype(str).str.len()

min_len = lengths.min()
max_len = lengths.max()

print(f"Min length: {min_len}")
print(f"Max length: {max_len}")

print("User IDs with the shortest length:")
print(devices[devices["user_id"].astype(str).str.len() == min_len]["user_id"].unique())
print("User IDs with the longest length:")
print(devices[devices["user_id"].astype(str).str.len() == max_len]["user_id"].unique())

Min length: 1
Max length: 5
User IDs with the shortest length:
<IntegerArray>
[1, 6, 4, 0, 5, 3, 7, 9, 8, 2]
Length: 10, dtype: Int64
User IDs with the longest length:
<IntegerArray>
[11885, 15533, 11964, 10670, 19220, 14587, 15593, 12295, 12617, 18270,
 ...
 13121, 15790, 10442, 15105, 10984, 13531, 12568, 12296, 19056, 12199]
Length: 9430, dtype: Int64


In [129]:
# Check if all user_id s are numeric
mask = notifs['user_id'].str.isnumeric()
all_numeric = mask.all()
all_numeric

True

In [130]:
# Then cast integer to the field
notifs['user_id'] = notifs['user_id'].astype('int64')

In [131]:
notifs.dtypes

reason                  object
channel                 object
status                  object
user_id                  int64
created_date    datetime64[ns]
dtype: object

In [132]:
#check for nulls
notifs.isnull().sum()

reason          0
channel         0
status          0
user_id         0
created_date    0
dtype: int64

In [133]:
#check for empty strings
(notifs=='').sum()

reason          0
channel         0
status          0
user_id         0
created_date    0
dtype: int64

In [134]:
#creates new clean file
notifs.to_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/notifications.parquet", index=False)

# Creating Expectations

In [135]:
context = gx.get_context()

## Devices

In [136]:
validator = context.sources.pandas_default.read_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/devices.parquet")

In [137]:
validator.expect_column_values_to_not_be_null("device_type")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [138]:
validator.expect_column_values_to_not_be_null("user_id")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [139]:
validator.expect_column_distinct_values_to_be_in_set(
    column="device_type",
    value_set=["Android","Apple","Unknown"]
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Android",
      "Apple",
      "Unknown"
    ],
    "details": {
      "value_counts": [
        {
          "value": "Android",
          "count": 9714
        },
        {
          "value": "Apple",
          "count": 9673
        },
        {
          "value": "Unknown",
          "count": 43
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [140]:
validator.expect_column_value_lengths_to_be_between(
    column="user_id",
    min_value=1,
    max_value=10
)


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Transactions

In [141]:
validator = context.sources.pandas_default.read_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/transactions.parquet")

In [142]:
for col in ["created_date", "user_id", "transaction_id","transactions_type","transactions_currency","amount_usd","transactions_state","ea_cardholderpresence","ea_merchant_country"]:
    result = validator.expect_column_values_to_not_be_null(col)
    print(f"Expectation for '{col}':", result)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'created_date': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "created_date",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'user_id': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_id",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Expectation for 'transactions_state': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "transactions_state",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'ea_cardholderpresence': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "ea_cardholderpresence",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'ea_merchant_country': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "ea_merchant_country",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [143]:
validator.expect_column_values_to_be_unique(column='transaction_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2740071,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [144]:
trans['created_date'].min()

Timestamp('2018-01-01 08:51:10.248709')

In [145]:
trans['created_date'].max()

Timestamp('2019-05-16 18:22:16.959896')

## Users

In [146]:
validator = context.sources.pandas_default.read_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/users.parquet")

In [147]:
for col in ["user_id", "birth_year","country","created_date","user_settings_crypto_unlocked","plan","num_contacts"]:
    result = validator.expect_column_values_to_not_be_null(col)
    print(f"Expectation for '{col}':", result)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'user_id': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_id",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'birth_year': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "birth_year",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Expectation for 'user_settings_crypto_unlocked': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_settings_crypto_unlocked",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'plan': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "plan",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'num_contacts': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "num_contacts",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [148]:
validator.expect_column_values_to_be_unique(column='user_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 19430,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Notifications

In [149]:
validator = context.sources.pandas_default.read_parquet("/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/notifications.parquet")

In [150]:
for col in ["reason", "channel","status","created_date","user_id"]:
    result = validator.expect_column_values_to_not_be_null(col)
    print(f"Expectation for '{col}':", result)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'reason': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "reason",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 121813,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'channel': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "channel",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 121813,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'status': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "status",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 121813,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'created_date': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "created_date",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 121813,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Expectation for 'user_id': {
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "user_id",
      "batch_id": "default_pandas_datasource-#ephemeral_pandas_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 121813,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


# Sending Cleaned Data to GCP Cloud Storage Bucket

In [157]:
from dotenv import load_dotenv

load_dotenv(override=True)

False

In [167]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/denizgokalp.5/.gcp_keys/mitchell5-key.json"


In [159]:
from google.cloud import storage

In [160]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to the GCS bucket."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Add filename if destination is a "folder"
    if destination_blob_name.endswith("/"):
        filename = os.path.basename(source_file_path)
        destination_blob_name += filename


    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)

    print(f"Uploaded {source_file_path} to gs://{bucket_name}/{destination_blob_name}")

In [168]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket("neobank_data_bucket_deniz")
blob = bucket.blob("test_upload_unique.txt")  # new name

blob.upload_from_string("Hello world!")
print("Upload successful")


Upload successful


In [163]:
from google.cloud import storage
client = storage.Client()
print(client.project)


tidal-vim-468818-s5


In [155]:
files_to_upload = [
    "transactions.parquet",
    "users.parquet",
    "notifications.parquet",
    "devices.parquet"
]

In [169]:
#for multiple files
for filename in files_to_upload:
    source_path = f"/home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/{filename}"
    upload_to_gcs(
        bucket_name="neobank_data_bucket_deniz",
        source_file_path=source_path,
        destination_blob_name="data/"
    )


Uploaded /home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/transactions.parquet to gs://neobank_data_bucket_deniz/data/transactions.parquet
Uploaded /home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/users.parquet to gs://neobank_data_bucket_deniz/data/users.parquet
Uploaded /home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/notifications.parquet to gs://neobank_data_bucket_deniz/data/notifications.parquet
Uploaded /home/denizgokalp.5/code/mitchell5/neo_bank/data/cleaned/devices.parquet to gs://neobank_data_bucket_deniz/data/devices.parquet


In [171]:
#checking it worked

client = storage.Client()
bucket = client.bucket("neobank_data_bucket_deniz")

blobs = list(bucket.list_blobs(prefix="data/"))

for blob in blobs:
    print(blob.name)


data/devices.parquet
data/notifications.parquet
data/transactions.parquet
data/users.parquet


In [172]:
!pwd

/home/denizgokalp.5/code/mitchell5/neo_bank/part_1-data_exploration


In [173]:
!which python

/home/denizgokalp.5/code/mitchell5/neo_bank/part_1-data_exploration/.venv/bin/python


# Sending it from Bucket to BigQuery

In [ ]:
#terminal commands

In [175]:
!gcloud config set project tidal-vim-468818-s5


Updated property [core/project].


In [177]:
!gcloud auth application-default set-quota-project tidal-vim-468818-s5



Credentials saved to file: [/home/denizgokalp.5/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "tidal-vim-468818-s5" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [179]:
!bq load --source_format=PARQUET \
--replace \
neobank_data_raw.raw_devices \
gs://neobank_data_bucket_deniz/data/devices.parquet


Waiting on bqjob_r5e91c118363ddaa1_00000198b9bdbb82_1 ... (1s) Current status: DONE   


In [180]:
!bq load --source_format=PARQUET \
--replace \
neobank_data_raw.raw_users \
gs://neobank_data_bucket_deniz/data/users.parquet

Waiting on bqjob_r7c300b0c588c92d1_00000198b9bdea14_1 ... (1s) Current status: DONE   


In [181]:
!bq load --source_format=PARQUET \
--replace \
neobank_data_raw.raw_transactions \
gs://neobank_data_bucket_deniz/data/transactions.parquet

Waiting on bqjob_r1bb70667ddae3d4c_00000198b9be0a48_1 ... (4s) Current status: DONE   


In [182]:
!bq load --source_format=PARQUET \
--replace \
neobank_data_raw.raw_notifications \
gs://neobank_data_bucket_deniz/data/notifications.parquet

Waiting on bqjob_r46a96226d1e1f5be_00000198b9be35ae_1 ... (1s) Current status: DONE   
